In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
root = Path('../output/runs')

In [ ]:
run = 'resnet50_lr_0.0001'
csv_path = str(root/run/Path('predict_test.csv'))
df = pd.read_csv(csv_path)
pred_counts = df['camera'].value_counts()
pred_counts /= 0.01 * df.shape[0]
pred_counts